In [2]:
# =============================================================================
# MODEL PERSISTENCE & TRAINING OPTIMIZATION
# =============================================================================
# Colab notebook for teaching saving, checkpoints, early stopping, overfitting
# Uses IMDB sentiment analysis throughout
# =============================================================================

# =============================================================================
# CELL 1: SETUP & LOAD DATA
# =============================================================================

!pip install -q tensorflow

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
import pickle
import os

print("✅ Libraries imported!")

# Load IMDB dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=5000)

# Use subset for faster training
x_train, y_train = x_train[:20000], y_train[:20000]
x_test, y_test = x_test[:5000], y_test[:5000]

# Pad sequences using Keras
x_train = keras.utils.pad_sequences(x_train, maxlen=100)
x_test = keras.utils.pad_sequences(x_test, maxlen=100)

# Create tokenizer from IMDB word_index
word_index = keras.datasets.imdb.get_word_index()
tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000)
tokenizer.word_index = word_index

# Helper to show reviews
index_to_word = {idx + 3: word for word, idx in word_index.items()}
index_to_word.update({0: '<PAD>', 1: '<START>', 2: '<UNKNOWN>'})

def show_review(index):
    """Show a review in readable text."""
    # Get original sequence from dataset
    (x_orig, y_orig), _ = keras.datasets.imdb.load_data(num_words=5000)
    text = ' '.join([index_to_word.get(num, '?') for num in x_orig[index]])
    sentiment = "POSITIVE 😊" if y_orig[index] == 1 else "NEGATIVE 😞"
    print(f"\n📝 Review #{index} ({sentiment})")
    print("-" * 50)
    print(text[:500] + "..." if len(text) > 500 else text)

print(f"\n✅ Data loaded!")
print(f"   Training samples: {len(x_train)}")
print(f"   Test samples: {len(x_test)}")
print(f"   Sequence length: {x_train.shape[1]} words")

print("\n📚 Sample reviews:")
show_review(0)
show_review(1)

✅ Libraries imported!

✅ Data loaded!
   Training samples: 20000
   Test samples: 5000
   Sequence length: 100 words

📚 Sample reviews:

📝 Review #0 (POSITIVE 😊)
--------------------------------------------------
<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNKNOWN> is an amazing actor and now the same being director <UNKNOWN> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNKNOWN> ...

📝 Review #1 (NEGATIVE 😞)
--------------------------------------------------
<START> big hair big <UNKNOWN> bad music and a giant safety <UNKNOWN> these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of

In [3]:
# =============================================================================
# CELL 2: THREE FORMATS EXPLAINED
# =============================================================================

print("""
=============================================================================
THREE WAYS TO SAVE A MODEL
=============================================================================

1. Keras Format (.keras) - RECOMMENDED
   └── my_model.keras          ← Everything in one file

   Use for: Default choice, full model save

2. H5 Format (.h5) - LEGACY
   └── my_model.h5             ← Everything in one file

   Use for: Older code compatibility

3. Weights Only (.weights.h5)
   └── my_weights.weights.h5   ← Just the learned numbers

   Use for: Transfer learning
   ⚠️  Must rebuild architecture before loading!

4. Export for Serving
   └── my_model_export/        ← Folder for TFServing/TFLite

   Use for: Production deployment

=============================================================================
""")



THREE WAYS TO SAVE A MODEL

1. Keras Format (.keras) - RECOMMENDED
   └── my_model.keras          ← Everything in one file

   Use for: Default choice, full model save

2. H5 Format (.h5) - LEGACY
   └── my_model.h5             ← Everything in one file

   Use for: Older code compatibility

3. Weights Only (.weights.h5)
   └── my_weights.weights.h5   ← Just the learned numbers

   Use for: Transfer learning
   ⚠️  Must rebuild architecture before loading!

4. Export for Serving
   └── my_model_export/        ← Folder for TFServing/TFLite

   Use for: Production deployment




In [4]:
# =============================================================================
# CELL 3: TRAIN MODEL
# =============================================================================

print("="*60)
print("TRAINING SENTIMENT MODEL")
print("="*60)

model = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100, name='embedding'),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2, name='lstm'),
    layers.Dense(1, activation='sigmoid', name='output')
], name='sentiment_model')

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("\n📊 Model Architecture:")
model.summary()

print("\n🏋️ Training...")
history = model.fit(
    x_train, y_train,
    epochs=3,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\n✅ Training complete! Test accuracy: {test_acc:.2%}")

TRAINING SENTIMENT MODEL

📊 Model Architecture:


/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/December/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sentiment_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


🏋️ Training...
Epoch 1/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6816 - loss: 0.5957 - val_accuracy: 0.7853 - val_loss: 0.4726
Epoch 2/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8296 - loss: 0.4026 - val_accuracy: 0.8295 - val_loss: 0.3814
Epoch 3/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8572 - loss: 0.3435 - val_accuracy: 0.8400 - val_loss: 0.3769

✅ Training complete! Test accuracy: 81.22%


In [5]:
# =============================================================================
# CELL 4: SAVE MODEL - 3 WAYS
# =============================================================================

print("="*60)
print("SAVING MODEL - THREE WAYS")
print("="*60)

os.makedirs('saved_models', exist_ok=True)

# Method 1: Keras format (recommended)
model.save('saved_models/sentiment_model.keras')
print("\n✅ Keras format saved: saved_models/sentiment_model.keras")

# Method 2: H5 format (legacy)
model.save('saved_models/sentiment_model.h5')
print("✅ H5 format saved: saved_models/sentiment_model.h5")

# Method 3: Weights only
model.save_weights('saved_models/sentiment_weights.weights.h5')
print("✅ Weights saved: saved_models/sentiment_weights.weights.h5")

# Method 4: Export for TFServing/TFLite
model.export('saved_models/sentiment_export')
print("✅ Exported: saved_models/sentiment_export/")

print("\n📁 Files created:")
!ls -lh saved_models/

SAVING MODEL - THREE WAYS

✅ Keras format saved: saved_models/sentiment_model.keras
✅ H5 format saved: saved_models/sentiment_model.h5
✅ Weights saved: saved_models/sentiment_weights.weights.h5
INFO:tensorflow:Assets written to: saved_models/sentiment_export/assets


INFO:tensorflow:Assets written to: saved_models/sentiment_export/assets


Saved artifact at 'saved_models/sentiment_export'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  4985454864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4980531152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4980539216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4985456784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4980539024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4985456592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4985457360: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Exported: saved_models/sentiment_export/

📁 Files created:
total 13336
drwxr-xr-x@ 6 kyle-anthonyhay  staff   192B Dec 30 11:56 sentiment_export
-rw-r--r--@ 1 kyle-anthonyhay  staff   2.0M Dec 30 11:56 sentiment_model.h5
-rw-r--r--@ 1 kyle-anthonyhay  st

In [6]:
# =============================================================================
# CELL 5: LOAD AND PREDICT
# =============================================================================

print("="*60)
print("LOADING MODEL & PREDICTING NEW TEXT")
print("="*60)

# Load model
loaded_model = keras.models.load_model('saved_models/sentiment_model.keras')
print("✅ Model loaded!")

def predict_sentiment(text):
    """Predict sentiment of any text."""
    sequence = tokenizer.texts_to_sequences([text])
    padded = keras.utils.pad_sequences(sequence, maxlen=100)
    prob = loaded_model.predict(padded, verbose=0)[0][0]
    return prob, "POSITIVE 😊" if prob > 0.5 else "NEGATIVE 😞"

# Test predictions
test_reviews = [
    "This movie was amazing! I loved it.",
    "Terrible film. Waste of time.",
    "It was okay, nothing special.",
]

print("\n🔮 Predictions:\n")
for review in test_reviews:
    prob, sentiment = predict_sentiment(review)
    print(f"📝 \"{review}\"")
    print(f"   → {prob:.1%} → {sentiment}\n")

LOADING MODEL & PREDICTING NEW TEXT
✅ Model loaded!

🔮 Predictions:

📝 "This movie was amazing! I loved it."
   → 73.2% → POSITIVE 😊

📝 "Terrible film. Waste of time."
   → 64.1% → POSITIVE 😊

📝 "It was okay, nothing special."
   → 54.3% → POSITIVE 😊



In [7]:
# =============================================================================
# CELL 6: SAVE/LOAD TOKENIZER
# =============================================================================

print("="*60)
print("SAVING TOKENIZER")
print("="*60)

# Save tokenizer
with open('saved_models/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
print("✅ Tokenizer saved!")

# Load tokenizer
with open('saved_models/tokenizer.pkl', 'rb') as f:
    loaded_tokenizer = pickle.load(f)
print("✅ Tokenizer loaded!")

# Test
test_text = "great movie"
print(f"\n📖 Test: \"{test_text}\" → {loaded_tokenizer.texts_to_sequences([test_text])}")

print("\n⚠️  Always save tokenizer with model!")

SAVING TOKENIZER
✅ Tokenizer saved!
✅ Tokenizer loaded!

📖 Test: "great movie" → [[84, 17]]

⚠️  Always save tokenizer with model!


In [8]:
# =============================================================================
# CELL 7: CHECKPOINT - SAVE BEST ONLY
# =============================================================================

print("="*60)
print("CHECKPOINT: SAVE BEST ONLY")
print("="*60)

os.makedirs('checkpoints', exist_ok=True)

model_ckpt = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation='sigmoid')
])
model_ckpt.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

checkpoint_best = ModelCheckpoint(
    'checkpoints/best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

print("🏋️ Training (watch 'saving model' messages)...\n")
model_ckpt.fit(
    x_train, y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.2,
    callbacks=[checkpoint_best]
)

print("\n✅ Best model saved: checkpoints/best_model.keras")

CHECKPOINT: SAVE BEST ONLY
🏋️ Training (watch 'saving model' messages)...

Epoch 1/5


/Users/kyle-anthonyhay/Documents/CODE/Revature-Training/Ai-Engineering/December/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


123/125 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6143 - loss: 0.6592
Epoch 1: val_accuracy improved from None to 0.78100, saving model to checkpoints/best_model.keras

Epoch 1: finished saving model to checkpoints/best_model.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.6924 - loss: 0.5982 - val_accuracy: 0.7810 - val_loss: 0.4571
Epoch 2/5
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8223 - loss: 0.4161
Epoch 2: val_accuracy improved from 0.78100 to 0.83650, saving model to checkpoints/best_model.keras

Epoch 2: finished saving model to checkpoints/best_model.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8268 - loss: 0.4049 - val_accuracy: 0.8365 - val_loss: 0.3734
Epoch 3/5
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8549 - loss: 0.3475
Epoch 3: val_accuracy did not improve from 0.83650
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8524 - loss: 0.3489 - val_accuracy: 0.8207 - val_loss: 0.4054
Epoch 4/5
123/125 ━━

In [9]:
# =============================================================================
# CELL 8: CHECKPOINT - SAVE EVERY EPOCH
# =============================================================================

print("="*60)
print("CHECKPOINT: SAVE EVERY EPOCH")
print("="*60)

model_all = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation='sigmoid')
])
model_all.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

checkpoint_all = ModelCheckpoint(
    'checkpoints/model_epoch{epoch:02d}_acc{val_accuracy:.3f}.keras',
    save_best_only=False,
    verbose=1
)

print("🏋️ Training (saving every epoch)...\n")
model_all.fit(
    x_train, y_train,
    epochs=3,
    batch_size=128,
    validation_split=0.2,
    callbacks=[checkpoint_all]
)

print("\n📁 All checkpoints:")
!ls -la checkpoints/model_epoch*

CHECKPOINT: SAVE EVERY EPOCH
🏋️ Training (saving every epoch)...

Epoch 1/3
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6269 - loss: 0.6510
Epoch 1: saving model to checkpoints/model_epoch01_acc0.811.keras

Epoch 1: finished saving model to checkpoints/model_epoch01_acc0.811.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.7049 - loss: 0.5807 - val_accuracy: 0.8108 - val_loss: 0.4264
Epoch 2/3
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8324 - loss: 0.3994
Epoch 2: saving model to checkpoints/model_epoch02_acc0.818.keras

Epoch 2: finished saving model to checkpoints/model_epoch02_acc0.818.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8347 - loss: 0.3921 - val_accuracy: 0.8183 - val_loss: 0.4031
Epoch 3/3
123/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8621 - loss: 0.3354
Epoch 3: saving model to checkpoints/model_epoch03_acc0.829.keras

Epoch 3: finished saving model to checkpoints/model_epoch03_acc0.829.keras
125/125 ━━━━━━

In [10]:
# =============================================================================
# CELL 9: RESUME TRAINING
# =============================================================================

print("="*60)
print("RESUME INTERRUPTED TRAINING")
print("="*60)

print("📂 Loading checkpoint...")
resumed_model = keras.models.load_model('checkpoints/best_model.keras')
print("✅ Loaded!")

print("\n🏋️ Resuming from epoch 3 to 6...")
resumed_model.fit(
    x_train, y_train,
    initial_epoch=3,
    epochs=6,
    batch_size=128,
    validation_split=0.2
)
print("\n✅ Training resumed!")

RESUME INTERRUPTED TRAINING
📂 Loading checkpoint...
✅ Loaded!

🏋️ Resuming from epoch 3 to 6...
Epoch 4/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.8536 - loss: 0.3549 - val_accuracy: 0.8298 - val_loss: 0.3939
Epoch 5/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8690 - loss: 0.3240 - val_accuracy: 0.8422 - val_loss: 0.3779
Epoch 6/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8851 - loss: 0.2923 - val_accuracy: 0.8367 - val_loss: 0.3893

✅ Training resumed!


In [11]:
# =============================================================================
# CELL 10: EARLY STOPPING
# =============================================================================

print("="*60)
print("EARLY STOPPING")
print("="*60)

model_es = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation='sigmoid')
])
model_es.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True,
    verbose=1
)

print("🏋️ Training with epochs=20 (early stopping will interrupt)...\n")
history_es = model_es.fit(
    x_train, y_train,
    epochs=20,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stop]
)

actual_epochs = len(history_es.history['loss'])
print(f"\n✅ Stopped at epoch {actual_epochs} (saved {20 - actual_epochs} epochs!)")

EARLY STOPPING
🏋️ Training with epochs=20 (early stopping will interrupt)...

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6883 - loss: 0.5948 - val_accuracy: 0.8033 - val_loss: 0.4380
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8294 - loss: 0.3990 - val_accuracy: 0.8267 - val_loss: 0.3848
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8587 - loss: 0.3411 - val_accuracy: 0.8215 - val_loss: 0.3961
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8767 - loss: 0.3040 - val_accuracy: 0.8202 - val_loss: 0.3958
Epoch 4: early stopping
Restoring model weights from the end of the best epoch: 2.

✅ Stopped at epoch 4 (saved 16 epochs!)


In [12]:
# =============================================================================
# CELL 11: REDUCE LR ON PLATEAU
# =============================================================================

print("="*60)
print("REDUCE LEARNING RATE ON PLATEAU")
print("="*60)

model_lr = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation='sigmoid')
])
model_lr.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=1,
    min_lr=0.00001,
    verbose=1
)

print("🏋️ Training (watch 'reducing learning rate' messages)...\n")
model_lr.fit(
    x_train, y_train,
    epochs=6,
    batch_size=128,
    validation_split=0.2,
    callbacks=[reduce_lr]
)

REDUCE LEARNING RATE ON PLATEAU
🏋️ Training (watch 'reducing learning rate' messages)...

Epoch 1/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6929 - loss: 0.5865 - val_accuracy: 0.7940 - val_loss: 0.4470 - learning_rate: 0.0010
Epoch 2/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8291 - loss: 0.3990 - val_accuracy: 0.8235 - val_loss: 0.4000 - learning_rate: 0.0010
Epoch 3/6
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8589 - loss: 0.3451 - val_accuracy: 0.8393 - val_loss: 0.3774 - learning_rate: 0.0010
Epoch 4/6
123/125 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8722 - loss: 0.3151
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8737 - loss: 0.3122 - val_accuracy: 0.8305 - val_loss: 0.3909 - learning_rate: 0.0010
Epoch 5/6
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8989 - loss: 0.2700
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.000250000

In [13]:
# =============================================================================
# CELL 12: ALL CALLBACKS TOGETHER
# =============================================================================

print("="*60)
print("PRODUCTION SETUP: ALL CALLBACKS")
print("="*60)

model_prod = keras.Sequential([
    layers.Embedding(5000, 32, input_length=100),
    layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(1, activation='sigmoid')
])
model_prod.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

callbacks_prod = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ModelCheckpoint('checkpoints/production_best.keras', monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, verbose=1)
]

print("🏋️ Training with all callbacks...\n")
model_prod.fit(
    x_train, y_train,
    epochs=15,
    batch_size=128,
    validation_split=0.2,
    callbacks=callbacks_prod
)

print("\n✅ Production training complete!")

PRODUCTION SETUP: ALL CALLBACKS
🏋️ Training with all callbacks...

Epoch 1/15
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5901 - loss: 0.6597
Epoch 1: val_accuracy improved from None to 0.79575, saving model to checkpoints/production_best.keras

Epoch 1: finished saving model to checkpoints/production_best.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.6833 - loss: 0.5955 - val_accuracy: 0.7958 - val_loss: 0.4532 - learning_rate: 0.0010
Epoch 2/15
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8286 - loss: 0.4089
Epoch 2: val_accuracy improved from 0.79575 to 0.82175, saving model to checkpoints/production_best.keras

Epoch 2: finished saving model to checkpoints/production_best.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8276 - loss: 0.4055 - val_accuracy: 0.8217 - val_loss: 0.3994 - learning_rate: 0.0010
Epoch 3/15
124/125 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8553 - loss: 0.3506
Epoch 3: val_accuracy improved from 0.821

In [14]:
# =============================================================================
# CELL 13: OVERFIT MODEL
# =============================================================================

print("="*60)
print("OVERFITTING DEMO: BIG MODEL, NO REGULARIZATION")
print("="*60)

model_overfit = keras.Sequential([
    layers.Embedding(5000, 128, input_length=100),
    layers.LSTM(128),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model_overfit.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Build model first to count parameters
model_overfit.build(input_shape=(None, 100))
print(f"📊 Parameters: {model_overfit.count_params():,}\n")

history_overfit = model_overfit.fit(
    x_train, y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.2
)

train_acc = history_overfit.history['accuracy'][-1]
val_acc = history_overfit.history['val_accuracy'][-1]
gap = train_acc - val_acc

print(f"\n📊 Train: {train_acc:.1%} | Val: {val_acc:.1%} | Gap: {gap:.1%}")
if gap > 0.1:
    print("⚠️  Gap > 10% = OVERFITTING!")

OVERFITTING DEMO: BIG MODEL, NO REGULARIZATION
📊 Parameters: 779,905

Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - accuracy: 0.7449 - loss: 0.5004 - val_accuracy: 0.8370 - val_loss: 0.3660
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.8743 - loss: 0.3044 - val_accuracy: 0.8510 - val_loss: 0.3632
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.8964 - loss: 0.2550 - val_accuracy: 0.8440 - val_loss: 0.3700
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - accuracy: 0.9236 - loss: 0.1963 - val_accuracy: 0.8380 - val_loss: 0.4322
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step - accuracy: 0.9426 - loss: 0.1494 - val_accuracy: 0.8410 - val_loss: 0.4811

📊 Train: 94.3% | Val: 84.1% | Gap: 10.2%
⚠️  Gap > 10% = OVERFITTING!
